In [125]:
import pandas as pd
import requests
import ast


# Pull in data from data.sfgov API

In [126]:
url = 'https://data.sfgov.org/resource/j2j3-acqj.json'
response = requests.get(url).json()
df = pd.DataFrame(response)
df.head()


,parcel_s,building_name,building_address,postal_code,full_address,property_type,pim_link,benchmark_2020_status,floor_area,year_built,...,_2012_weather_normalized_site_eui_kbtu_ft2,_2012_weather_normalized_source_eui_kbtu_sq_ft,_2012_percent_better_than_national_median_source_eui,_2011_energy_star_score,_2011_site_eui_kbtu_ft2,_2011_percent_better_than_national_median_site_eui,_2011_weather_normalized_site_eui_kbtu_ft2,_2011_source_eui_kbtu_ft2,_2011_percent_better_than_national_median_source_eui,_2011_weather_normalized_source_eui_kbtu_ft2
0,0022/014,2627 TAYLOR ST: Cannabis,2627 Taylor St,94133,"{'latitude': '37.806603', 'longitude': '-122.4...",Cannabis Business,{'url': 'http://propertymap.sfplanning.org/?&s...,Upcoming,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0065/015,901 COLUMBUS AVE,901 COLUMBUS AVE,94133,"{'latitude': '37.803053', 'longitude': '-122.4...",Mixed Residential,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,99333,1916,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0105/019,350 UNION ST,350 UNION ST,94133,"{'latitude': '37.80093', 'longitude': '-122.40...",Mixed Residential,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Insufficient Data,85460,1965,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0158/047,1520 TAYLOR ST,1520 TAYLOR ST,94133,"{'latitude': '37.796419', 'longitude': '-122.4...",Mixed Residential,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,67734,1976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0175/028,435 PACIFIC AVE,435 PACIFIC AVE,94133,"{'latitude': '37.797422', 'longitude': '-122.4...",Commercial,{'url': 'http://propertymap.sfplanning.org/?&s...,Violation - Did Not Report,33652,2002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Fix two issues

### Split latitude, longitude, and human-readable address from full_address

In [127]:
df = pd.concat([df.drop(['full_address'], axis=1), df['full_address'].apply(pd.Series)], axis=1)
pd.concat([df.iloc[:, :5], df.iloc[:, -4:]], axis=1).head()


,parcel_s,building_name,building_address,postal_code,property_type,latitude,longitude,human_address,0
0,0022/014,2627 TAYLOR ST: Cannabis,2627 Taylor St,94133,Cannabis Business,37.806603,-122.415489,"{""address"": ""2627 Taylor St"", ""city"": ""San Fra...",NaN
1,0065/015,901 COLUMBUS AVE,901 COLUMBUS AVE,94133,Mixed Residential,37.803053,-122.414162,"{""address"": ""901 COLUMBUS AVE"", ""city"": ""SAN F...",NaN
2,0105/019,350 UNION ST,350 UNION ST,94133,Mixed Residential,37.80093,-122.405254,"{""address"": ""350 UNION ST"", ""city"": ""SAN FRANC...",NaN
3,0158/047,1520 TAYLOR ST,1520 TAYLOR ST,94133,Mixed Residential,37.796419,-122.413447,"{""address"": ""1520 TAYLOR ST"", ""city"": ""SAN FRA...",NaN
4,0175/028,435 PACIFIC AVE,435 PACIFIC AVE,94133,Commercial,37.797422,-122.402743,"{""address"": ""435 PACIFIC AVE"", ""city"": ""SAN FR...",NaN


Drop NaN column

In [128]:
df.drop(0, axis=1, inplace=True)
pd.concat([df.iloc[:, :5], df.iloc[:, -3:]], axis=1).head()


,parcel_s,building_name,building_address,postal_code,property_type,latitude,longitude,human_address
0,0022/014,2627 TAYLOR ST: Cannabis,2627 Taylor St,94133,Cannabis Business,37.806603,-122.415489,"{""address"": ""2627 Taylor St"", ""city"": ""San Fra..."
1,0065/015,901 COLUMBUS AVE,901 COLUMBUS AVE,94133,Mixed Residential,37.803053,-122.414162,"{""address"": ""901 COLUMBUS AVE"", ""city"": ""SAN F..."
2,0105/019,350 UNION ST,350 UNION ST,94133,Mixed Residential,37.80093,-122.405254,"{""address"": ""350 UNION ST"", ""city"": ""SAN FRANC..."
3,0158/047,1520 TAYLOR ST,1520 TAYLOR ST,94133,Mixed Residential,37.796419,-122.413447,"{""address"": ""1520 TAYLOR ST"", ""city"": ""SAN FRA..."
4,0175/028,435 PACIFIC AVE,435 PACIFIC AVE,94133,Commercial,37.797422,-122.402743,"{""address"": ""435 PACIFIC AVE"", ""city"": ""SAN FR..."


Do the same with the resulting human_address... which is actually a string, not a dict :(

In [129]:
df = pd.concat([df.drop(['human_address'], axis=1), df['human_address'].fillna(value='1').apply(lambda x: ast.literal_eval(x)).apply(pd.Series)], axis=1)
pd.concat([df.iloc[:, :5], df.iloc[:, -5:]], axis=1).head()


,parcel_s,building_name,building_address,postal_code,property_type,address,city,state,zip,0
0,0022/014,2627 TAYLOR ST: Cannabis,2627 Taylor St,94133,Cannabis Business,2627 Taylor St,San Francisco,CA,94133,NaN
1,0065/015,901 COLUMBUS AVE,901 COLUMBUS AVE,94133,Mixed Residential,901 COLUMBUS AVE,SAN FRANCISCO,CA,94133,NaN
2,0105/019,350 UNION ST,350 UNION ST,94133,Mixed Residential,350 UNION ST,SAN FRANCISCO,CA,94133,NaN
3,0158/047,1520 TAYLOR ST,1520 TAYLOR ST,94133,Mixed Residential,1520 TAYLOR ST,SAN FRANCISCO,CA,94133,NaN
4,0175/028,435 PACIFIC AVE,435 PACIFIC AVE,94133,Commercial,435 PACIFIC AVE,SAN FRANCISCO,CA,94133,NaN


Drop NaN column (again) and zip (since postal_code already exists)

In [130]:
df.drop(0, axis=1, inplace=True)
df.drop('zip', axis=1, inplace=True)
pd.concat([df.iloc[:, :5], df.iloc[:, -5:]], axis=1).head()


,parcel_s,building_name,building_address,postal_code,property_type,latitude,longitude,address,city,state
0,0022/014,2627 TAYLOR ST: Cannabis,2627 Taylor St,94133,Cannabis Business,37.806603,-122.415489,2627 Taylor St,San Francisco,CA
1,0065/015,901 COLUMBUS AVE,901 COLUMBUS AVE,94133,Mixed Residential,37.803053,-122.414162,901 COLUMBUS AVE,SAN FRANCISCO,CA
2,0105/019,350 UNION ST,350 UNION ST,94133,Mixed Residential,37.80093,-122.405254,350 UNION ST,SAN FRANCISCO,CA
3,0158/047,1520 TAYLOR ST,1520 TAYLOR ST,94133,Mixed Residential,37.796419,-122.413447,1520 TAYLOR ST,SAN FRANCISCO,CA
4,0175/028,435 PACIFIC AVE,435 PACIFIC AVE,94133,Commercial,37.797422,-122.402743,435 PACIFIC AVE,SAN FRANCISCO,CA


### Change all columns (except state) to Title Case

In [131]:
df = pd.concat([df.iloc[:, :-1].apply(lambda x: x.str.title()), df.iloc[:, -1:]], axis=1)
pd.concat([df.iloc[:, :5], df.iloc[:, -5:]], axis=1).head()

,parcel_s,building_name,building_address,postal_code,property_type,latitude,longitude,address,city,state
0,0022/014,2627 Taylor St: Cannabis,2627 Taylor St,94133,Cannabis Business,37.806603,-122.415489,2627 Taylor St,San Francisco,CA
1,0065/015,901 Columbus Ave,901 Columbus Ave,94133,Mixed Residential,37.803053,-122.414162,901 Columbus Ave,San Francisco,CA
2,0105/019,350 Union St,350 Union St,94133,Mixed Residential,37.80093,-122.405254,350 Union St,San Francisco,CA
3,0158/047,1520 Taylor St,1520 Taylor St,94133,Mixed Residential,37.796419,-122.413447,1520 Taylor St,San Francisco,CA
4,0175/028,435 Pacific Ave,435 Pacific Ave,94133,Commercial,37.797422,-122.402743,435 Pacific Ave,San Francisco,CA
